In [1]:
%load_ext Cython

In [3]:
%%cython --compile-args=-fopenmp --link-args=-fopenmp --force
# cython : boundscheck = False
cimport numpy as np
from cython.parallel import prange
import time
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import minmax_scale
from sklearn.datasets import make_classification
import multiprocessing
from libc.stdlib cimport malloc, free

global X,Y
global classes, num_features, num_samples, num_relevant_features
cpdef int cpu_count
cpdef int num_threads

cpdef make_dataset(int num_classes, int num_features, int num_samples, int useful_features):
    x,y = make_classification(n_samples=num_samples,n_classes=num_classes,n_informative=useful_features,n_features=num_features)
    return x,y 

cdef double pearson_correlation(double *X, int *Y, int n) nogil:
    cdef double sig_x = 0.0, sig_y = 0.0 ,sig_xy = 0.0, sig_x2 = 0.0, sig_y2 = 0.0
    for i in range(n):
        sig_x += X[i]
        sig_y += Y[i]
        sig_xy += X[i]*Y[i]
        sig_x2 += (X[i]*X[i])
        sig_y2 += (Y[i]*Y[i])
    cdef double ans =((n*sig_xy -sig_x*sig_y)/(((n*sig_x2-sig_x**2)*(n*sig_y2-sig_y**2))**0.5))
    if ans < 0 :
        ans *= -1
    return ans

cdef double pearson_correlation2(double *X, double *Y, int n) nogil:
    cdef double sig_x = 0.0, sig_y = 0.0 ,sig_xy = 0.0, sig_x2 = 0.0, sig_y2 = 0.0
    for i in range(n):
        sig_x += X[i]
        sig_y += Y[i]
        sig_xy += X[i]*Y[i]
        sig_x2 += (X[i]*X[i])
        sig_y2 += (Y[i]*Y[i])
    cdef double ans =((n*sig_xy -sig_x*sig_y)/(((n*sig_x2-sig_x**2)*(n*sig_y2-sig_y**2))**0.5))
    if ans < 0 :
        ans *= -1
    return ans

cdef double Redundancy(int target, int *S, double **redundancy, int ls) nogil:
    cdef double ans = 0.0
    cdef int i
    for i in range(ls):
        ans+= redundancy[target][S[i]]
    return ans
    
    
cpdef select_k_features(np.ndarray[double, ndim=2, mode='c'] x_train, np.ndarray y_train, int k):
    cdef double* relevance = <double*> malloc((num_features)* sizeof(double)) # edit : num_features+1
    cdef double **x_train_ = <double**>malloc((num_features)*sizeof(double*))
    cdef double **redundancy = <double**> malloc((num_features)*sizeof(double*))
    cdef int *y_train_ = <int*> y_train.data
    cdef int *S = <int*> malloc((k)*sizeof(int*)) #stores the set of all features currently selected
    cdef int i
    for i in range(num_features):
        x_train_[i] = &x_train[i,0]
        redundancy[i] = <double*> malloc ((num_features)*sizeof(double))
    cdef int num_feature = x_train.shape[0]
    cdef int n = x_train.shape[1]
    cdef int num_thread = num_threads
    with nogil: # parallel calculation of relevances
        for i in prange(num_feature, schedule='guided', num_threads=num_thread):
            relevance[i] = (pearson_correlation(x_train_[i],y_train_,n))
    feature_list = []
    cdef int max_relevance = 0
    for i in range(1,num_feature): #select the one with the highest relevance
        if relevance[i] > relevance[max_relevance]:
            max_relevance = i
    feature_list.append(max_relevance)
    S[0] = max_relevance
    with nogil:
        for i in prange(num_feature,schedule='guided',num_threads=num_thread):
            redundancy[i][max_relevance] = pearson_correlation2(x_train_[i],x_train_[max_relevance],n)
            redundancy[max_relevance][i] = redundancy[i][max_relevance]
    cdef double total_relevance = relevance[max_relevance]
    cdef double total_redundancy = 0.0
    cdef int f,l,it,ptr,feature
    cdef int* imp_features_list = <int*> malloc((num_feature)*sizeof(int))
    cdef double* imp_features = <double*> malloc((num_feature)*sizeof(double))
    
    for i in range(1,k):
        l = num_feature - i # possible candidates  
        ptr = 0
        for f in range(num_feature):
            if f not in feature_list:
                imp_features_list[ptr] = f
                ptr = ptr+1


        with nogil:
             for it in prange(l, schedule = 'guided', num_threads = num_thread):
                feature = imp_features_list[it]
                imp_features[it] = (total_relevance*i+relevance[feature])/(i+1)
                imp_features[it]-= (total_redundancy*i*i+Redundancy(feature,S,redundancy,i))/((i+1)**2)
        max_relevance = 0
       
        for it in range(1,l):
            if imp_features[it] > imp_features[max_relevance]:
                max_relevance = it
        
        S[i] = imp_features_list[max_relevance]
        total_redundancy = (total_redundancy*i*i+Redundancy(imp_features_list[max_relevance],S,redundancy,i))/((i+1)**2)
        total_relevance = (total_relevance*i + relevance[imp_features_list[max_relevance]])/(i+1)
        feature_list.append(imp_features_list[max_relevance])
        max_relevance = imp_features_list[max_relevance]
        with nogil:
            for it in prange(num_feature,schedule='guided',num_threads=num_thread):
                redundancy[it][max_relevance] = pearson_correlation2(x_train_[it],x_train_[max_relevance],n)
                redundancy[max_relevance][it] = redundancy[it][max_relevance]
    return feature_list
    
num_classes = 50
num_features = 100
num_samples = 5000
useful_features = 84
X,Y = make_classification(n_samples=num_samples,n_classes=num_classes,n_informative=useful_features,n_features=num_features) # Synthesis of dataset
x_train,x_test,y_train,y_test = train_test_split(X, Y, test_size = 0.25,random_state = 42)
cpu_count = multiprocessing.cpu_count()
num_threads = cpu_count
n = len(x_train)    
k = int(0.2*num_features) # number of features to be selected
print(x_train.T.shape)
t1 = time.time()
selected_features  = select_k_features(x_train.T.copy(order='c'),y_train.T.copy(order='c'),k)
print("Time",time.time()-t1)
print(selected_features)

(100, 3750)
Time 0.031332969665527344
[37, 27, 80, 61, 47, 51, 10, 28, 54, 25, 4, 63, 26, 14, 53, 93, 6, 42, 11, 96]
